# Getting started with AutoML Tables

To use this Colab notebook, copy it to your own Google Drive and open it with [Colaboratory](https://colab.research.google.com/) (or Colab). To run a cell hold the Shift key and press the Enter key (or Return key). Colab automatically displays the return value of the last line in each cell. Refer to [this page](https://colab.research.google.com/notebooks/welcome.ipynb) for more information on Colab.

You can run a Colab notebook on a hosted runtime in the Cloud. The hosted VM times out after 90 minutes of inactivity and you will lose all the data stored in the memory including your authentication data. If your session gets disconnected (for example, because you closed your laptop) for less than the 90 minute inactivity timeout limit, press 'RECONNECT' on the top right corner of your notebook and resume the session. After Colab timeout, you'll need to

1.   Re-run the initialization and authentication.
2.   Continue from where you left off. You may need to copy-paste the value of some variables such as the `dataset_name` from the printed output of the previous cells.

Alternatively you can connect your Colab notebook to a [local runtime](https://research.google.com/colaboratory/local-runtimes.html).



## 1. Project set up





Follow the [AutoML Tables documentation](https://cloud.google.com/automl-tables/docs/) to
* Create a Google Cloud Platform (GCP) project.
* Enable billing.
* Apply to whitelist your project.
* Enable AutoML API.
* Enable AutoML Tables API.
* Create a service account, grant required permissions, and download the service account private key.

You also need to upload your data into Google Cloud Storage (GCS) or BigQuery. For example, to use GCS as your data source
* Create a GCS bucket.
* Upload the training and batch prediction files.


**Warning:** Private keys must be kept secret. If you expose your private key it is recommended to revoke it immediately from the Google Cloud Console.



---



## 2. Initialize and authenticate
This section runs intialization and authentication. It creates an authenticated session which is required for running any of the following sections.

### Install the client library
Run the following cell.

In [ ]:
#@title Install AutoML Tables client library { vertical-output: true }
!pip install google-cloud-automl

Enter your GCP project ID.

In [ ]:
#@title GCP project ID and location

project_id = 'lwiots' #@param {type:'string'}
region = 'us-central1'

### Authenticate using service account key
Run the following cell. Click on the 'Choose Files' button and select the service account private key file. If your Service Account key file or folder is hidden, you can reveal it in a Mac by pressing the <b>Command + Shift + .</b> combo.

In [ ]:
#@title Authenticate and create a client. { vertical-output: true }

from google.colab import files
from google.cloud import automl_v1beta1
from google.oauth2 import service_account

# Upload service account key
keyfile_upload = files.upload()
keyfile_name = list(keyfile_upload.keys())[0]

# Authenticate and create an AutoML client.
client = automl_v1beta1.TablesClient(
    project=project_id,
    region=region,
    credentials=service_account.Credentials.from_service_account_file(keyfile_name)
)

### Test

To test whether your project set up and authentication steps were successful, run the following cell to list your datasets.

In [ ]:
#@title List datasets. { vertical-output: true }

list_datasets_response = client.list_datasets()
datasets = {
    dataset.display_name: dataset.name for dataset in list_datasets_response
}
datasets

You can also print the list of your models by running the following cell.

In [ ]:
#@title List models. { vertical-output: true }

list_models_response = client.list_models()
models = {model.display_name: model.name for model in list_models_response}
models



---



## 3. Import training data

### Create dataset

Select a dataset display name and pass your table source information to create a new dataset.

In [ ]:
#@title Create dataset { vertical-output: true, output-height: 200 }

dataset_display_name = 'test_census_income_pred' #@param {type: 'string'}

dataset = client.create_dataset(dataset_display_name)
dataset

### Import data

You can import your data to AutoML Tables from GCS or BigQuery. For this tutorial, you can use the [census_income dataset](https://storage.cloud.google.com/cloud-ml-data/automl-tables/notebooks/census_income.csv) 
as your training data. You can create a GCS bucket and upload the  data intofa your bucket. The URI for your file is `gs://BUCKET_NAME/FOLDER_NAME1/FOLDER_NAME2/.../FILE_NAME`. Alternatively you can create a BigQuery table and upload the data into the table. The URI for your table is `bq://PROJECT_ID.DATASET_ID.TABLE_ID`.

Importing data may take a few minutes or hours depending on the size of your data. If your Colab times out, run the following command to retrieve your dataset. Replace `dataset_name` with its actual value obtained in the preceding cells.

    dataset = client.get_dataset(dataset_name)

In [ ]:
#@title ... if data source is GCS { vertical-output: true }

gcs_input_uris = ['gs://cloud-ml-data/automl-tables/notebooks/census_income.csv',] #@param
import_data_response = client.import_data(dataset=dataset, gcs_input_uris=gcs_input_uris)

In [ ]:
#@title ... if data source is BigQuery { vertical-output: true }

bigquery_input_uri = 'bq://my-project-trial5.census_income.income_census' #@param {type: 'string'}
import_data_response = client.import_data(dataset=dataset, bigquery_input_uris=bigquery_input_uris)

In [ ]:
 #@title Import data { vertical-output: true }

print('Dataset import operation: {}'.format(import_data_response.operation))
# Wait until import is done.
import_data_result = import_data_response.result()
import_data_result

### Review the specs

Run the following command to see table specs such as row count.

In [ ]:
#@title Table schema { vertical-output: true }

import google.cloud.automl_v1beta1.proto.data_types_pb2 as data_types
import matplotlib.pyplot as plt

# List table specs
list_table_specs_response = client.list_table_specs(dataset=dataset)
table_specs = [s for s in list_table_specs_response]
# List column specs
table_spec_name = table_specs[0].name
list_column_specs_response = client.list_column_specs(dataset=dataset)
column_specs = {s.display_name: s for s in list_column_specs_response}
# Table schema pie chart.
type_counts = {}
for column_spec in column_specs.values():
  type_name = data_types.TypeCode.Name(column_spec.data_type.type_code)
  type_counts[type_name] = type_counts.get(type_name, 0) + 1

plt.pie(x=type_counts.values(), labels=type_counts.keys(), autopct='%1.1f%%')
plt.axis('equal')
plt.show()


Run the following command to see column specs such inferred schema.

___

## 4. Update dataset: assign a label column and enable nullable columns

AutoML Tables automatically detects your data column type. For example, for the ([census_income](https://storage.cloud.google.com/cloud-ml-data/automl-tables/notebooks/census_income.csv)) it detects `income` to be categorical (as it is just either over or under 50k) and `age` to be numerical. Depending on the type of your label column, AutoML Tables chooses to run a classification or regression model. If your label column contains only numerical values, but they represent categories, change your label column type to categorical by updating your schema.

### Update a column: set to nullable

In [ ]:
#@title Update dataset { vertical-output: true }

client.update_column_spec(
    dataset=dataset, 
    column_spec_display_name='income', 
    type_code='CATEGORY'
)

**Tip:** You can use `type_code='CATEGORY'` to convert the column data type from `FLOAT64` to `CATEGORY`.

### Update dataset: assign a label

In [ ]:
#@title Update dataset { vertical-output: true }

client.set_target_column(dataset=dataset, 
                         column_spec_display_name='income')

___

## 5. Creating a model

### Train a model
Specify the duration of the training. For example, `'train_budget_milli_node_hours': 1000` runs the training for one hour. If your Colab times out, use `client.list_models(location_path)` to check whether your model has been created. Then use model name to continue to the next steps. Run the following command to retrieve your model. Replace `model_name` with its actual value.

    model = client.get_model(model_name)

In [ ]:
#@title Create model { vertical-output: true }

model_display_name = 'census_income_model' #@param {type:'string'}

create_model_response = client.create_model(
    model_display_name, 
    dataset=dataset, 
    train_budget_milli_node_hours=1000
)

print('Dataset import operation: {}'.format(create_model_response.operation))

# Wait until model training is done.
model = create_model_response.result()
model

___

## 6. Make a prediction

### There are two different prediction modes: online and batch. The following cells show you how to make an online prediction. 

Run the following cell, and then choose the desired test values for your online prediction.

In [ ]:
#@title Make an online prediction: set the categorical variables{ vertical-output: true }
from ipywidgets import interact
import ipywidgets as widgets

workclass_ids = ['Private', 'Self-emp-not-inc', 'Self-emp-inc', 'Federal-gov', 'Local-gov', 'State-gov', 'Without-pay', 'Never-worked']
education_ids = ['Bachelors', 'Some-college', '11th', 'HS-grad', 'Prof-school', 'Assoc-acdm', 'Assoc-voc', '9th', '7th-8th', '12th', 'Masters', '1st-4th', '10th', 'Doctorate', '5th-6th', 'Preschool']
marital_status_ids = ['Married-civ-spouse', 'Divorced', 'Never-married', 'Separated', 'Widowed', 'Married-spouse-absent', 'Married-AF-spouse']
occupation_ids = ['Tech-support', 'Craft-repair', 'Other-service', 'Sales', 'Exec-managerial', 'Prof-specialty', 'Handlers-cleaners', 'Machine-op-inspct', 'Adm-clerical', 'Farming-fishing', 'Transport-moving', 'Priv-house-serv', 'Protective-serv', 'Armed-Forces']
relationship_ids = ['Wife', 'Own-child', 'Husband', 'Not-in-family', 'Other-relative', 'Unmarried']
race_ids = ['White', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other', 'Black']
sex_ids = ['Female', 'Male']
native_country_ids = ['United-States', 'Cambodia', 'England', 'Puerto-Rico', 'Canada', 'Germany', 'Outlying-US(Guam-USVI-etc)', 'India', 'Japan', 'Greece', 'South', 'China', 'Cuba', 'Iran', 'Honduras', 'Philippines', 'Italy', 'Poland', 'Jamaica', 'Vietnam', 'Mexico', 'Portugal', 'Ireland', 'France', 'Dominican-Republic', 'Laos', 'Ecuador', 'Taiwan', 'Haiti', 'Columbia', 'Hungary', 'Guatemala', 'Nicaragua', 'Scotland', 'Thailand', 'Yugoslavia', 'El-Salvador', 'Trinadad&Tobago', 'Peru', 'Hong', 'Holand-Netherlands']
workclass = widgets.Dropdown(options=workclass_ids, value=workclass_ids[0],
                             description='workclass:')

education = widgets.Dropdown(options=education_ids, value=education_ids[0],
                             description='education:', width='500px')

marital_status = widgets.Dropdown(options=marital_status_ids, value=marital_status_ids[0],
                                  description='marital status:', width='500px')

occupation = widgets.Dropdown(options=occupation_ids, value=occupation_ids[0],
                              description='occupation:', width='500px')

relationship = widgets.Dropdown(options=relationship_ids, value=relationship_ids[0],
                                description='relationship:', width='500px')

race = widgets.Dropdown(options=race_ids, value=race_ids[0],
                        description='race:', width='500px')

sex = widgets.Dropdown(options=sex_ids, value=sex_ids[0],
                       description='sex:', width='500px')

native_country = widgets.Dropdown(options=native_country_ids, value=native_country_ids[0],
                                  description='native_country:', width='500px')

display(workclass)
display(education)
display(marital_status)
display(occupation)
display(relationship)
display(race)
display(sex)
display(native_country)


Adjust the slides on the right to the desired test values for your online prediction.

In [ ]:
#@title Make an online prediction: set the numeric variables{ vertical-output: true }

age = 34 #@param {type:'slider', min:1, max:100, step:1}
capital_gain = 40000 #@param {type:'slider', min:0, max:100000, step:10000}
capital_loss = 3.8 #@param {type:'slider', min:0, max:4000, step:0.1}
fnlwgt = 150000 #@param {type:'slider', min:0, max:1000000, step:50000}
education_num = 9 #@param {type:'slider', min:1, max:16, step:1}
hours_per_week = 40 #@param {type:'slider', min:1, max:100, step:1}


In [ ]:
response = client.deploy_model(model=model)
response.result()


Run the prediction, only after the model finishes deployment

In [ ]:
inputs = {
    'age': age,
    'workclass': workclass.value,
    'fnlwgt': fnlwgt,
    'education': education.value,
    'education_num': education_num,
    'marital_status': marital_status.value,
    'occupation': occupation.value,
    'relationship': relationship.value,
    'race': race.value,
    'sex': sex.value,
    'capital_gain': capital_gain,
    'capital_loss': capital_loss,
    'hours_per_week': hours_per_week,
    'native_country': native_country.value
}

prediction_client.predict(model=model, inputs=inputs)

Undeploy the model

In [ ]:
response = client.undeploy_model(model=model)

## 7. Batch prediction

### Initialize prediction

Your data source for batch prediction can be GCS or BigQuery. For this tutorial, you can use [census_income_batch_prediction_input.csv](https://storage.cloud.google.com/cloud-ml-data/automl-tables/notebooks/census_income_batch_prediction_input.csv) as input source. Create a GCS bucket and upload the file into your bucket. Some of the lines in the batch prediction input file are intentionally left missing some values. The AutoML Tables logs the errors in the `errors.csv` file.
Also, enter the UI and create the bucket into which you will load your predictions. The bucket's default name here is automl-tables-pred.

**NOTE:** The client library has a bug. If the following cell returns a `TypeError: Could not convert Any to BatchPredictResult` error, ignore it. The batch prediction output file(s) will be updated to the GCS bucket that you set in the preceding cells.

In [ ]:
#@title Start batch prediction { vertical-output: true, output-height: 200 }

batch_predict_gcs_input_uris = ['gs://cloud-ml-data/automl-tables/notebooks/census_income_batch_prediction_input.csv',] #@param
batch_predict_gcs_output_uri_prefix = 'gs://automl-tables-pred1' #@param {type:'string'}

batch_predict_response = prediction_client.batch_predict(
    model=model, 
    gcs_input_uris=gcs_input_uris, 
    gcs_output_uri_prefix=gcs_output_uri_prefix
)

print('Batch prediction operation: {}'.format(batch_predict_response.operation))
# Wait until batch prediction is done.
batch_predict_result = batch_predict_response.result()
batch_predict_response.metadata